In [1]:
# %% test cuda
import torch #type: ignore
torch.cuda.is_available()

True

In [2]:
# %% update data
import socket
from scripts.util.data.DataUpdater import update_server_main , update_laptop_main
from scripts.util.data.sqlConnector import update_sql_since , update_sql_dates

if socket.gethostname() == 'mengkjin-server':
    update_server_main()
else:
    update_laptop_main()
update_sql_since()
if False:
    update_sql_dates(20240101 , 20240229)

Use device name: NVIDIA GeForce RTX 4090
Merge Update Files
['/home/mengkjin/Workspace/SharedFolder/DB_updater.0.h5']
Sat Mar  2 08:55:19 2024 : DB_updater.0.h5 > data/DB_data/DB_information/DB_information.h5/./basic/calendar copying Done!
Sat Mar  2 08:55:28 2024 : DB_updater.0.h5 > data/DB_data/DB_information/DB_information.h5/./stock/concepts copying Done!
Sat Mar  2 08:55:29 2024 : DB_updater.0.h5 > data/DB_data/DB_information/DB_information.h5/./stock/description copying Done!
Sat Mar  2 08:55:47 2024 : DB_updater.0.h5 > data/DB_data/DB_information/DB_information.h5/./stock/industry copying Done!
Sat Mar  2 08:55:47 2024 : DB_updater.0.h5 > data/DB_data/DB_information/DB_information.h5/./stock/st copying Done!
Sat Mar  2 08:55:47 2024 : DB_updater.0.h5 > data/DB_data/DB_labels/DB_labels.2024.h5/./10days/lag0 copying Done!
Sat Mar  2 08:55:47 2024 : DB_updater.0.h5 > data/DB_data/DB_labels/DB_labels.2024.h5/./10days/lag1 copying Done!
Sat Mar  2 08:55:47 2024 : DB_updater.0.h5 > da

In [1]:
# %% test a specific model
from run_model import main
main(process = 0 , rawname = 1 , resume = 0 , anchoring = 0)

24-02-24 08:07:35|MOD:run_model   |: Start Process [Load Data]!


--Process Queue : Data + Train + Test + Instance
--Start Training New!
--Model_name is set to resnet_lstm_30m_ShortTest!
Load  2 DataBlocks ...... cost 30.86 secs
Align 2 DataBlocks ...... cost 14.22 secs


24-02-24 08:08:29|MOD:run_model   |: Finish Process [Load Data]! Cost 53.3Secs
24-02-24 08:08:29|MOD:run_model   |: Start Process [Train Model]!


Pre-Norming method of [30m] : [endpoint_division(False) , history_standardize(True)]


resnet_lstm_30m_ShortTest #0 @20170103 LoadData Cost    3.8Secs


score function of [pearson] calculated and success!
loss function of [pearson] calculated and success!


FirstBite Ep#  0 : loss  1.00047, train-0.00043, valid 0.01743, max 0.0174, best 0.0174, lr1.0e-07
FirstBite Ep#  5 : loss  0.83565, train 0.17961, valid 0.18999, max 0.1900, best 0.1900, lr3.8e-03
FirstBite Ep# 10 : loss  0.80110, train 0.22181, valid 0.21142, max 0.2114, best 0.2114, lr1.3e-03
FirstBite Ep# 15 : loss  0.76813, train 0.26382, valid 0.23901, max 0.2390, best 0.2390, lr6.3e-04
24-02-24 08:09:36|MOD:run_model   |: resnet_lstm_30m_ShortTest #0 @20170103|Round 0 FirstBite Ep# 19 Max Epoch|Train 0.2717 Valid 0.2443 BestVal 0.2443|Cost  1.1Min,  3.1Sec/Ep
FirstBite Ep#  0 : loss  1.01057, train-0.01047, valid-0.01367, max-0.0137, best-0.0137, lr1.0e-07
FirstBite Ep#  5 : loss  0.84296, train 0.17090, valid 0.17953, max 0.1795, best 0.1795, lr3.8e-03
FirstBite Ep# 10 : loss  0.80822, train 0.21297, valid 0.20051, max 0.2005, best 0.2005, lr1.3e-03
FirstBite Ep# 15 : loss  0.77852, train 0.25041, valid 0.22327, max 0.2233, best 0.2233, lr6.3e-04
24-02-24 08:11:03|MOD:run_model

In [5]:
import os
del_path = './instance/tra_lstm2_day_ShortTest'
if os.path.exists(del_path): os.rmdir(del_path)
# rm -r ./instance/tra_lstm2_day_ShortTest #type: ignore

In [ ]:
# %% model data preprocessing
from data_preprocessing import main
main(confirm=1)